In [49]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

In [50]:
potential = 'AHO'
cutoff = 8
shots = 1024

In [51]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [52]:
min_3_ev = eigenvalues.argsort()[:3]
min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[1]])

In [53]:
operator_pool = []
for i in range(num_qubits):
    #operator_pool.append(qml.Identity(i))
    #operator_pool.append(qml.PauliX(i))
    operator_pool.append(qml.PauliY(i))
    operator_pool.append(qml.PauliZ(i))


In [54]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
basis_state = [0,0,0,0]

@qml.qnode(dev)
def circuit(times, op_list, try_cz=False, cz_wires=None, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))

    for i, op in enumerate(op_list):
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    if try_cz:
        qml.CZ(wires=cz_wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [55]:
def cost_function(times, op_list):

    times = pnp.tensor(times, requires_grad=True)
    energy = circuit(times, op_list)
    
    return energy

In [56]:
dev2 = qml.device("default.qubit", wires=(2*num_qubits + 1), shots=shots)
@qml.qnode(dev2)
def swap_test(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state):

    trotter_steps=1
    bs = (basis_state + basis_state)
    qml.BasisState(bs, wires=range(2*num_qubits))

    ancilla = 2*num_qubits
    qml.Hadamard(wires=ancilla)

    prev_param_index = 0
    for pop in prev_op_list:

        if type(pop) == qml.CZ:
            qml.CZ(wires=pop.wires)
        else:
            qml.ApproxTimeEvolution(pop, time=prev_op_params[prev_param_index], n=trotter_steps)
            prev_param_index += 1
    

    param_index  = 0
    for op in op_list:
        if type(op) == qml.CZ:
            w0 = op.wires[0] + num_qubits
            w1 = op.wires[1] + num_qubits
            wire_map = {op.wires[0]:w0, op.wires[1]:w1}
            op = op.map_wires(wire_map)
            qml.CZ(wires=op.wires)
        else:
            wire = op.wires[0] + num_qubits
            wire_map = {op.wires[0]: wire}
            op = op.map_wires(wire_map)
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)  
            param_index += 1

    qml.ApproxTimeEvolution(trial_op, time=3.14, n=trotter_steps)

    qml.Barrier()
    for i in range(num_qubits):
        qml.CSWAP(wires=[ancilla, i, num_qubits + i])

    qml.Hadamard(wires=ancilla)

    prob = qml.probs(wires=ancilla)

    return prob

In [57]:
def multi_swap_test(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state, P=30):
    
        results = []
        for _ in range(P):
            prob = swap_test(trial_op, op_list, op_params, prev_op_list, prev_op_params, basis_state)
            results.append(prob[0])
        
        avg_prob = sum(results) / P
        overlap = 2 * avg_prob - 1

        return overlap

In [58]:
dev3 = qml.device("default.qubit", wires=num_qubits, shots=shots)
@qml.qnode(dev3)
def grad_circuit(times, trial_op, op_list, op_params, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))
    param_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)
            param_index +=1

    qml.ApproxTimeEvolution(trial_op, time=times, n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [59]:
def compute_grad(trial_op, op_list, op_params):
    t = pnp.tensor(0.0, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(t, trial_op, op_list, op_params)
    return grad

In [60]:
def grad_plus_overlap(trial_op, op_list, op_params, prev_op_list, prev_op_params, beta=20.0):

    grad = compute_grad(trial_op, op_list, op_params)
    
    penalty = 0
    if len(prev_op_list) != 0:
        for prev_op, prev_param in zip(prev_op_list, prev_op_params):
            ol = multi_swap_test(trial_op, op_list, op_params, prev_op, prev_param, basis_state)
            penalty += (beta*ol)
            if penalty > 0:
                print(f"Applying penalty {penalty} to op {trial_op} with gradient {abs(grad)}")
    
    return abs(grad), penalty

In [61]:
num_levels = 2
num_steps = 4
prev_op_list = []
prev_op_params = []

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

for e_level in range(num_levels):

    print("######################################")
    print(f"Energy level: {e_level}")

    op_list = []
    op_params = []

    for i in range(num_steps):

        print(f"step: {i}")

        grad_list = []
        overlaps = []

        for trial_op in operator_pool:
            grad, ol = grad_plus_overlap(trial_op, op_list, op_params, prev_op_list, prev_op_params)
            grad_list.append(grad)
            overlaps.append(ol)

        norm_grad = [i/sum(grad_list) for i in grad_list]
        results = [a - b for a, b in zip(norm_grad, overlaps)]
        maxidx = np.argmax(grad_list)
        #maxidx = np.argmin(grad_list)
        op_list.append(operator_pool[maxidx])

        print(f"Best op is: {operator_pool[maxidx]}")

        # Generate Halton sequence
        num_dimensions = len(op_list)
        num_samples = popsize
        halton_sampler = Halton(d=num_dimensions)
        halton_samples = halton_sampler.random(n=num_samples)
        scaled_samples = 2 * np.pi * halton_samples

        bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
        
        res = differential_evolution(cost_function,
                                        bounds,
                                        args=(op_list,),
                                        maxiter=max_iter,
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy,
                                        popsize=popsize,
                                        init=scaled_samples,
                                        )
        
        if i!=0: 
            pre_min_e = min_e
            pre_op_params = op_params

        min_e = res.fun
        op_params = res.x

        print(f"Min E: {min_e}")
        print(res.success)

        if i!=0:
            if abs(pre_min_e - min_e) < 1e-8:
                print("gradient converged")
                op_list.pop()
                op_params = pre_op_params
                print(op_list, op_params)
                prev_op_list.append(op_list.copy())
                prev_op_params.append(op_params.copy())
                break


    

######################################
Energy level: 0
step: 0
Best op is: Y(2)
Min E: 1.7702455168368003
False
step: 1
Best op is: Y(1)
Min E: 1.6801555245826594
True
step: 2
Best op is: Y(2)
Min E: 1.6801555245826594
True
gradient converged
[Y(2), Y(1)] [2.78239457 3.29592164]
######################################
Energy level: 1
step: 0
Applying penalty 17.055989583333332 to op Y(0) with gradient 0.33770378749238117
Applying penalty 17.186197916666668 to op Z(0) with gradient 0.08789821023056454
Applying penalty 17.045572916666668 to op Y(1) with gradient 6.637286507785959
Applying penalty 16.971354166666668 to op Z(1) with gradient 0.8914404881891893
Applying penalty 17.0625 to op Y(2) with gradient 15.688352246617267
Applying penalty 17.03515625 to op Z(2) with gradient 1.1139906382212548
Applying penalty 17.102864583333336 to op Y(3) with gradient 0.5728334567567579
Applying penalty 17.05078125 to op Z(3) with gradient 0.15755285760564552
Best op is: Y(2)
Min E: 1.75638551802847

In [42]:
np.sort(eigenvalues)

array([3.20101100e-02+0.j, 1.68015552e+00+0.j, 1.83352558e+00+0.j,
       3.96337391e+00+0.j, 4.00985462e+00+0.j, 4.03692369e+00+0.j,
       8.26209394e+00+0.j, 8.77647491e+00+0.j, 4.25280809e+01+0.j,
       4.34333354e+01+0.j, 5.53066896e+01+0.j, 5.62063259e+01+0.j,
       3.81878539e+02+0.j, 3.82821281e+02+0.j, 4.08644293e+02+0.j,
       4.09587044e+02+0.j])

In [ ]:
prev_op_list

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.state()

In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

In [ ]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [ ]:
bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]

# Generate Halton sequence
num_dimensions = len(op_list)
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-6
abs_tol = 1e-5
popsize = 20

print("Running for overlap")
overlap_res = differential_evolution(overlap_function,
                                    bounds,
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )

In [ ]:
overlap_res.fun

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def energy_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [ ]:
x0 = overlap_res.x
x0
energy_circuit(x0)

In [ ]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [ ]:
data

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

num_steps = 5
op_list = []
op_params = []

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]
    res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    